<a href="https://colab.research.google.com/github/YanggWendy/FYP-Portfolio-Management-System/blob/main/FYP_2000_2022_123.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'FYP Data/'
assert FOLDERNAME is not None, "[1]Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


In [ ]:
!pip install yfinance
!pip install fundamentalanalysis
PATH = '/content/drive/My Drive/FYP Data/' 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import yfinance as yf
import pandas as pd
from tqdm.notebook import tqdm
import statistics
import fundamentalanalysis as fa
import datetime
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau,TensorBoard
from tensorflow.keras.models import load_model
import math
from keras import Input # for instantiating a keras tensor
from keras.layers import Bidirectional, GRU, RepeatVector, Dense, TimeDistributed # for creating layers inside the Neural Network
from keras.optimizers import SGD

In [ ]:
sp500url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
data_table = pd.read_html(sp500url)
data_table[0]
SP500_ticker = data_table[0]['Symbol'].tolist()

#remove invalid stock ticker
SP500_ticker.remove('BRK.B')
SP500_ticker.remove('BF.B')
SP500_ticker.remove('GEHC')
#SP500_ticker.remove('BG')
#SP500_ticker.remove('FICO')
#SP500_ticker.remove('PODD')

In [ ]:
Technical_list = ['Close','Volume','VEMA12','VSTD20','V20','AR','BR','AU','AD']
fundamental_list = ['marketCap', 'peRatio', 'pbRatio', 'evToSales','receivablesTurnover', 'payablesTurnover','debtToAssets', 'inventoryTurnover',
 'roe','revenuePerShare','cashPerShare']

#Load data

In [ ]:
stock_data = pd.read_csv(("drive/My Drive/FYP Data/price.csv"),index_col = [0], header = [0,1])
stock_data

ALGN                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
2023-01-31  265.260010  269.779999  265.000000  269.730011  269.730011   
2023-01-30  263.500000  268.549988  261.500000  265.820007  265.820007   
2023-01-27  260.000000  270.000000  259.019989  269.200012  269.200012   
2023-01-26  255.860001  263.100006  254.990005  262.950012  262.950012   
2023-01-25  246.820007  252.470001  245.639999  252.080002  252.080002   
...                ...         ...         ...         ...         ...   
1993-02-04         NaN         NaN         NaN         NaN         NaN   
1993-02-03         NaN         NaN         NaN         NaN         NaN   
1993-02-02         NaN         NaN         NaN         NaN         NaN   
1993-02-01         NaN         NaN         NaN         NaN         NaN   
1993-01-29         NaN         NaN         NaN         NaN         NaN   

                             SBAC                                      ...  \
               Volume        Open        High         Low       Close  ...   
Date                                                                   ...   
2023-01-31   844700.0  291.950012  297.730011  289.690002  297.529999  ...   
2023-01-30  1454300.0  288.920013  295.019989  287.100006  290.200012  ...   
2023-01-27  1165100.0  289.190002  291.970001  286.890015  289.029999  ...   
2023-01-26  1177000.0  292.429993  294.559998  288.970001  291.079987  ...   
2023-01-25   820700.0  289.140015  292.380005  286.160004  291.649994  ...   
...               ...         ...         ...         ...         ...  ...   
1993-02-04        NaN         NaN         NaN         NaN         NaN  ...   
1993-02-03        NaN         NaN         NaN         NaN         NaN  ...   
1993-02-02        NaN         NaN         NaN         NaN         NaN  ...   
1993-02-01        NaN         NaN         NaN         NaN         NaN  ...   
1993-01-29        NaN         NaN         NaN         NaN         NaN  ...   

                   XOM                                           DD  \
                   Low       Close   Adj Close    Volume       Open   
Date                                                                  
2023-01-31  110.430000  116.010002  115.124138  27861800  72.910004   
2023-01-30  113.150002  113.559998  112.692841  18672100  73.099998   
2023-01-27  115.389999  115.610001  114.727188  15179200  73.059998   
2023-01-26  114.339996  117.760002  116.860771  21019700  73.169998   
2023-01-25  111.389999  113.209999  112.345512  11914700  73.790001   
...                ...         ...         ...       ...        ...   
1993-02-04   15.375000   15.500000    5.945010   4631600  25.559553   
1993-02-03   15.406250   15.625000    5.923919   5699600  25.737461   
1993-02-02   15.343750   15.562500    5.900226   3359600  25.263039   
1993-02-01   15.281250   15.375000    5.829138   3186400  25.322342   
1993-01-29   15.156250   15.281250    5.793594   4369600  24.670010   

                                                                   
                 High        Low      Close  Adj Close     Volume  
Date                                                               
2023-01-31  74.000000  72.269997  73.949997  73.584908  2736000.0  
2023-01-30  73.559998  72.449997  72.650002  72.291336  2542000.0  
2023-01-27  74.190002  73.059998  73.550003  73.186890  2171300.0  
2023-01-26  73.790001  72.099998  73.730003  73.366005  3134500.0  
2023-01-25  74.180000  72.980003  73.529999  73.166985  3193100.0  
...               ...        ...        ...        ...        ...  
1993-02-04  26.211885  25.559553  26.033976   9.430413  1581710.0  
1993-02-03  25.915371  25.381643  25.500250   9.237073  1414770.0  
1993-02-02  25.737461  25.263039  25.678158   9.301520   895825.0  
1993-02-01  25.440947  25.085129  25.322342   9.172627   943461.0  
1993-01-2

In [ ]:
label_df = pd.read_csv("drive/My Drive/FYP Data/label.csv",index_col = 0)
label_df = label_df.reindex(index=label_df.index[::-1])
label_df.reset_index()
label_df = label_df[84:]
label_df

,MMM,AOS,ABT,ABBV,ACN,ATVI,ADM,ADBE,ADP,AAP,...,WTW,GWW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-31,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2000-02-29,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
2000-03-31,0,1,1,0,0,0,0,1,1,0,...,0,0,0,1,0,1,1,0,0,0
2000-04-28,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2000-05-31,0,0,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,0,0,1,1,0,1,0,0,0,0,...,1,0,1,0,1,1,0,1,0,1
2022-09-30,0,0,0,0,0,0,1,0,0,1,...,0,1,0,0,1,0,0,0,0,0
2022-10-31,0,1,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0


In [ ]:
#read data
multi_factor_data = pd.read_csv("drive/My Drive/FYP Data/multi_factor_data_improved.csv",index_col = 0,header = [0,1])
multi_factor_data[np.isnan(multi_factor_data)] = 0
multi_factor_data = multi_factor_data[84:]
#get time list
time = multi_factor_data.index.values
time

array(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
       '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
       '2000-09-29', '2000-10-31', '2000-11-30', '2000-12-29',
       '2001-01-31', '2001-02-28', '2001-03-30', '2001-04-30',
       '2001-05-31', '2001-06-29', '2001-07-31', '2001-08-31',
       '2001-09-28', '2001-10-31', '2001-11-30', '2001-12-31',
       '2002-01-31', '2002-02-28', '2002-03-28', '2002-04-30',
       '2002-05-31', '2002-06-28', '2002-07-31', '2002-08-30',
       '2002-09-30', '2002-10-31', '2002-11-29', '2002-12-31',
       '2003-01-31', '2003-02-28', '2003-03-31', '2003-04-30',
       '2003-05-30', '2003-06-30', '2003-07-31', '2003-08-29',
       '2003-09-30', '2003-10-31', '2003-11-28', '2003-12-31',
       '2004-01-30', '2004-02-27', '2004-03-31', '2004-04-30',
       '2004-05-28', '2004-06-30', '2004-07-30', '2004-08-31',
       '2004-09-30', '2004-10-29', '2004-11-30', '2004-12-31',
       '2005-01-31', '2005-02-28', '2005-03-31', '2005-

#Prepare Dataset

In [ ]:
# get multifactor list
multifactor_list = Technical_list+fundamental_list

# get x_dataset, y_dataset
multi_factor_data = multi_factor_data.astype(float)
label_df = label_df.astype(float)
x_dataset = []
y_dataset = []

for ticker in tqdm(SP500_ticker):
  y_dataset.append(label_df[ticker])
  #print(ticker)
  for factor in multifactor_list:
    x_dataset.append(multi_factor_data[ticker, factor])

y_dataset = np.array(y_dataset).T
x_dataset = np.array(x_dataset).T

print('y_dataset',y_dataset.shape)
print('x_dataset',x_dataset.shape)

  0%|          | 0/500 [00:00<?, ?it/s]

y_dataset (276, 500)
x_dataset (276, 10000)


In [ ]:
n_future = 1   # Number of month we want top predict into the future
n_past = 3  

In [ ]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
x_dataset = scaler.fit_transform(x_dataset)
y_dataset = scaler.fit_transform(y_dataset)
x_dataset.shape

(276, 10000)

In [ ]:
#Preparation of training set
X = []
Y = []

for i in range(n_past, len(x_dataset) - n_future +1):
  #print(i)
  X.append(x_dataset[i - n_past:i,:])
Y = y_dataset[n_past:]
X, Y = np.array(X), np.array(Y)
print(X.shape)
print(Y.shape)

(273, 3, 10000)
(273, 500)


In [ ]:
train_point = X.shape[0]-26
train_point

247

In [ ]:
#Preparation of test set
x_train = X[0:train_point]
y_train = Y[0:train_point]

x_test = X[train_point:]
y_test = Y[train_point:]

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(247, 3, 10000)
(247, 500)
(26, 3, 10000)
(26, 500)


#LSTM

In [ ]:
# Initializing the Neural Network based on LSTM
model = Sequential()

# Adding 1st LSTM layer
model.add(LSTM(units=1024, return_sequences=True, input_shape=(n_past, x_dataset.shape[1])))

# Adding 2nd LSTM layer
model.add(LSTM(units=1024, return_sequences=False))

# Adding Dropout
model.add(Dropout(0.2))

# Output layer
model.add(Dense(units=512, activation='linear'))

# Output layer
model.add(Dense(units=500, activation='linear'))

# Compiling the Neural Network
model.compile(optimizer = Adam(learning_rate=0.01), loss='mean_squared_error')
#model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=10, verbose=1)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
mcp = ModelCheckpoint(filepath='/content/drive/My Drive/FYP Data/weights_w_fundamental_1.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False)

tb = TensorBoard('logs')

history = model.fit(x_train, y_train, shuffle=True, epochs=10, callbacks=[es, rlr, mcp, tb], validation_split=0.2, verbose=1, batch_size=10)

Epoch 1/10
14/20 [====================>.........] - ETA: 0s - loss: 29.1203
Epoch 1: val_loss improved from inf to 9.08739, saving model to /content/drive/My Drive/FYP Data/weights_w_fundamental_1.h5
20/20 [==============================] - 7s 224ms/step - loss: 25.6217 - val_loss: 9.0874 - lr: 0.0100
Epoch 2/10
15/20 [=====================>........] - ETA: 0s - loss: 5.3655
Epoch 2: val_loss improved from 9.08739 to 0.56369, saving model to /content/drive/My Drive/FYP Data/weights_w_fundamental_1.h5
20/20 [==============================] - 2s 93ms/step - loss: 4.5484 - val_loss: 0.5637 - lr: 0.0100
Epoch 3/10
15/20 [=====================>........] - ETA: 0s - loss: 1.3031
Epoch 3: val_loss improved from 0.56369 to 0.45926, saving model to /content/drive/My Drive/FYP Data/weights_w_fundamental_1.h5
20/20 [==============================] - 4s 214ms/step - loss: 1.2116 - val_loss: 0.4593 - lr: 0.0100
Epoch 4/10
15/20 [=====================>........] - ETA: 0s - loss: 0.6330
Epoch 4: val_

In [ ]:
#Model Evaluation
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print('rmse', rmse)

#profit test
test_period = time[n_past+train_point:]

#calculate the top 10 stock we picked
stock_selection_index = np.argsort(predictions)[:,-30:]
stock_selection = []
for index in stock_selection_index:
  stock_list = []
  for stock_i in index:
    stock_list.append(SP500_ticker[stock_i])
  stock_selection.append(stock_list)

#trading simulate
original_money = 0
trading = 0
for stock in stock_selection[0]:
  original_money = original_money + stock_data[stock]["Close"][test_period[0]]
  #print(stock_data[stock]["Close"][test_period[0]])


trading = - original_money
i = 0

for stock_list in stock_selection:
  if i==0 or 25:
    i = i + 1
    continue

  for stock in stock_list:
    trading = trading + stock_data[stock]["Close"][test_period[i-1]] - stock_data[stock]["Close"][test_period[i]]
    #print(trading)
  
  i = i + 1

end_money = 0

for stock in stock_selection[-1]:
  trading = trading + stock_data[stock]["Close"][test_period[-1]]
  end_money = end_money + stock_data[stock]["Close"][test_period[-1]]
  #print(stock_data[stock]["Close"][test_period[-1]])

print('The money earned by LSTM with Fundamental and Technical data is ',trading)
print('The money earned by LSTM with Fundamental and Technical data is %',trading*100/original_money)
print('The money earned by LSTM with Fundamental and Technical data is %(unchange)',(end_money-original_money)*100/original_money)

1/1 [==============================] - 1s 648ms/step
rmse 0.011940589107573032
The money earned by LSTM with Fundamental and Technical data is  1704.2148513793945
The money earned by LSTM with Fundamental and Technical data is % 27.10763248665312
The money earned by LSTM with Fundamental and Technical data is %(unchange) 27.10763248665312


#GRU

In [ ]:
%%time
# The GRU architecture
regressorGRU = Sequential()
# First GRU layer with Dropout regularisation
regressorGRU.add(GRU(units=1024, return_sequences=True, input_shape=(n_past, x_dataset.shape[1]), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Second GRU layer
regressorGRU.add(GRU(units=512, return_sequences=True, input_shape=(n_past, x_dataset.shape[1]), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Third GRU layer
regressorGRU.add(GRU(units=500, return_sequences=True, input_shape=(n_past, x_dataset.shape[1]), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Fourth GRU layer
regressorGRU.add(GRU(units=500, activation='tanh'))
regressorGRU.add(Dropout(0.2))
# The output layer
regressorGRU.add(Dense(units=500))

# The output layer
regressorGRU.add(Dense(units=500))

# Compiling the RNN
regressorGRU.compile(optimizer = Adam(learning_rate=0.01), loss='mean_squared_error')
#regressorGRU.compile(optimizer=SGD(lr=0.01, decay=1e-7, momentum=0.9, nesterov=False),loss='mean_squared_error')

es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=10, verbose=1)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
mcp = ModelCheckpoint(filepath='/content/drive/My Drive/FYP Data/weights_GRU.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False)

tb = TensorBoard('logs')

# Fitting to the training set
history = regressorGRU.fit(x_train, y_train, shuffle=True, epochs=10, callbacks=[es, rlr, mcp, tb], validation_split=0.2, verbose=1, batch_size=10)

Epoch 1/10
20/20 [==============================] - ETA: 0s - loss: 10.9014
Epoch 1: val_loss improved from inf to 0.38808, saving model to /content/drive/My Drive/FYP Data/weights_GRU.h5
20/20 [==============================] - 11s 291ms/step - loss: 10.9014 - val_loss: 0.3881 - lr: 0.0100
Epoch 2/10
19/20 [===========================>..] - ETA: 0s - loss: 2.6001
Epoch 2: val_loss did not improve from 0.38808
20/20 [==============================] - 0s 11ms/step - loss: 2.5317 - val_loss: 1.8783 - lr: 0.0100
Epoch 3/10
15/20 [=====================>........] - ETA: 0s - loss: 1.3784
Epoch 3: val_loss improved from 0.38808 to 0.32041, saving model to /content/drive/My Drive/FYP Data/weights_GRU.h5
20/20 [==============================] - 1s 72ms/step - loss: 1.3407 - val_loss: 0.3204 - lr: 0.0100
Epoch 4/10
14/20 [====================>.........] - ETA: 0s - loss: 1.1432
Epoch 4: val_loss did not improve from 0.32041
20/20 [==============================] - 0s 11ms/step - loss: 1.0297 - 

In [ ]:
#Model Evaluation
predictions = regressorGRU.predict(x_test)
predictions = scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print('rmse', rmse)

#profit test
test_period = time[n_past+245:]

#calculate the top 10 stock we picked
stock_selection_index = np.argsort(predictions)[:,-30:]
stock_selection = []
for index in stock_selection_index:
  stock_list = []
  for stock_i in index:
    stock_list.append(SP500_ticker[stock_i])
  stock_selection.append(stock_list)

#trading simulate
original_money = 0
trading = 0
for stock in stock_selection[0]:
  original_money = original_money + stock_data[stock]["Close"][test_period[0]]
  #print(stock_data[stock]["Close"][test_period[0]])


trading = - original_money
i = 0

for stock_list in stock_selection:
  if i==0 or 25:
    i = i + 1
    continue

  for stock in stock_list:
    trading = trading + stock_data[stock]["Close"][test_period[i-1]] - stock_data[stock]["Close"][test_period[i]]
    #print(trading)
  
  i = i + 1

end_money = 0

for stock in stock_selection[-1]:
  trading = trading + stock_data[stock]["Close"][test_period[-1]]
  end_money = end_money + stock_data[stock]["Close"][test_period[-1]]
  #print(stock_data[stock]["Close"][test_period[-1]])

print('The money earned by LSTM with Fundamental and Technical data is ',trading)
print('The money earned by LSTM with Fundamental and Technical data is %',trading*100/original_money)
print('The money earned by LSTM with Fundamental and Technical data is %(unchange)',(end_money-original_money)*100/original_money)

1/1 [==============================] - 1s 1s/step
rmse 0.025873210886063484
The money earned by LSTM with Fundamental and Technical data is  660.0535297393799
The money earned by LSTM with Fundamental and Technical data is % 15.001348028290357
The money earned by LSTM with Fundamental and Technical data is %(unchange) 15.001348028290357


#Train with past 5 years

In [ ]:
n_future = 1   # Number of month we want top predict into the future
n_past = 5  

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
x_dataset = scaler.fit_transform(x_dataset)
y_dataset = scaler.fit_transform(y_dataset)

#Preparation of training set
X = []
Y = []

for i in range(n_past, len(x_dataset) - n_future +1):
  #print(i)
  X.append(x_dataset[i - n_past:i,:])
Y = y_dataset[n_past:]
X, Y = np.array(X), np.array(Y)
print(X.shape)
print(Y.shape)

(271, 5, 10000)
(271, 500)


In [ ]:
train_point = X.shape[0]-26

#Preparation of test set
x_train = X[0:train_point]
y_train = Y[0:train_point]

x_test = X[train_point:]
y_test = Y[train_point:]

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(245, 5, 10000)
(245, 500)
(26, 5, 10000)
(26, 500)


#LSTM

In [ ]:
# Initializing the Neural Network based on LSTM
model = Sequential()

# Adding 1st LSTM layer
model.add(LSTM(units=1024, return_sequences=True, input_shape=(n_past, x_dataset.shape[1])))

# Adding 2nd LSTM layer
model.add(LSTM(units=1024, return_sequences=False))

# Adding Dropout
model.add(Dropout(0.2))

# Output layer
model.add(Dense(units=512, activation='linear'))

# Output layer
model.add(Dense(units=500, activation='linear'))

# Compiling the Neural Network
model.compile(optimizer = Adam(learning_rate=0.01), loss='mean_squared_error')
#model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=10, verbose=1)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
mcp = ModelCheckpoint(filepath='/content/drive/My Drive/FYP Data/weights_w_fundamental_1.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False)

tb = TensorBoard('logs')

history = model.fit(x_train, y_train, shuffle=True, epochs=10, callbacks=[es, rlr, mcp, tb], validation_split=0.2, verbose=1, batch_size=10)

#Model Evaluation
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print('rmse', rmse)

#profit test
test_period = time[n_past+train_point:]

#calculate the top 10 stock we picked
stock_selection_index = np.argsort(predictions)[:,-30:]
stock_selection = []
for index in stock_selection_index:
  stock_list = []
  for stock_i in index:
    stock_list.append(SP500_ticker[stock_i])
  stock_selection.append(stock_list)

#trading simulate
original_money = 0
trading = 0
for stock in stock_selection[0]:
  original_money = original_money + stock_data[stock]["Close"][test_period[0]]
  #print(stock_data[stock]["Close"][test_period[0]])


trading = - original_money
i = 0

for stock_list in stock_selection:
  if i==0 or 25:
    i = i + 1
    continue

  for stock in stock_list:
    trading = trading + stock_data[stock]["Close"][test_period[i-1]] - stock_data[stock]["Close"][test_period[i]]
    #print(trading)
  
  i = i + 1

end_money = 0

for stock in stock_selection[-1]:
  trading = trading + stock_data[stock]["Close"][test_period[-1]]
  end_money = end_money + stock_data[stock]["Close"][test_period[-1]]
  #print(stock_data[stock]["Close"][test_period[-1]])

print('The money earned by LSTM with Fundamental and Technical data is ',trading)
print('The money earned by LSTM with Fundamental and Technical data is %',trading*100/original_money)
print('The money earned by LSTM with Fundamental and Technical data is %(unchange)',(end_money-original_money)*100/original_money)

Epoch 1/10
14/20 [====================>.........] - ETA: 0s - loss: 13.4692
Epoch 1: val_loss improved from inf to 0.44177, saving model to /content/drive/My Drive/FYP Data/weights_w_fundamental_1.h5
20/20 [==============================] - 8s 292ms/step - loss: 12.1928 - val_loss: 0.4418 - lr: 0.0100
Epoch 2/10
19/20 [===========================>..] - ETA: 0s - loss: 4.6328
Epoch 2: val_loss did not improve from 0.44177
20/20 [==============================] - 0s 12ms/step - loss: 4.6507 - val_loss: 1.4699 - lr: 0.0100
Epoch 3/10
14/20 [====================>.........] - ETA: 0s - loss: 1.1958
Epoch 3: val_loss improved from 0.44177 to 0.38359, saving model to /content/drive/My Drive/FYP Data/weights_w_fundamental_1.h5
20/20 [==============================] - 2s 92ms/step - loss: 1.0972 - val_loss: 0.3836 - lr: 0.0100
Epoch 4/10
15/20 [=====================>........] - ETA: 0s - loss: 0.5284
Epoch 4: val_loss improved from 0.38359 to 0.23352, saving model to /content/drive/My Drive/FYP

#GRU

In [ ]:
%%time
# The GRU architecture
regressorGRU = Sequential()
# First GRU layer with Dropout regularisation
regressorGRU.add(GRU(units=1024, return_sequences=True, input_shape=(n_past, x_dataset.shape[1]), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Second GRU layer
regressorGRU.add(GRU(units=512, return_sequences=True, input_shape=(n_past, x_dataset.shape[1]), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Third GRU layer
regressorGRU.add(GRU(units=500, return_sequences=True, input_shape=(n_past, x_dataset.shape[1]), activation='tanh'))
regressorGRU.add(Dropout(0.2))
# Fourth GRU layer
regressorGRU.add(GRU(units=500, activation='tanh'))
regressorGRU.add(Dropout(0.2))
# The output layer
regressorGRU.add(Dense(units=500))

# The output layer
regressorGRU.add(Dense(units=500))

# Compiling the RNN
regressorGRU.compile(optimizer = Adam(learning_rate=0.01), loss='mean_squared_error')
#regressorGRU.compile(optimizer=SGD(lr=0.01, decay=1e-7, momentum=0.9, nesterov=False),loss='mean_squared_error')

es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=10, verbose=1)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
mcp = ModelCheckpoint(filepath='/content/drive/My Drive/FYP Data/weights_GRU.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False)

tb = TensorBoard('logs')

# Fitting to the training set
history = regressorGRU.fit(x_train, y_train, shuffle=True, epochs=10, callbacks=[es, rlr, mcp, tb], validation_split=0.2, verbose=1, batch_size=10)


#Model Evaluation
predictions = regressorGRU.predict(x_test)
predictions = scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print('rmse', rmse)

#profit test
test_period = time[n_past+245:]

#calculate the top 10 stock we picked
stock_selection_index = np.argsort(predictions)[:,-30:]
stock_selection = []
for index in stock_selection_index:
  stock_list = []
  for stock_i in index:
    stock_list.append(SP500_ticker[stock_i])
  stock_selection.append(stock_list)

#trading simulate
original_money = 0
trading = 0
for stock in stock_selection[0]:
  original_money = original_money + stock_data[stock]["Close"][test_period[0]]
  #print(stock_data[stock]["Close"][test_period[0]])


trading = - original_money
i = 0

for stock_list in stock_selection:
  if i==0 or 25:
    i = i + 1
    continue

  for stock in stock_list:
    trading = trading + stock_data[stock]["Close"][test_period[i-1]] - stock_data[stock]["Close"][test_period[i]]
    #print(trading)
  
  i = i + 1

end_money = 0

for stock in stock_selection[-1]:
  trading = trading + stock_data[stock]["Close"][test_period[-1]]
  end_money = end_money + stock_data[stock]["Close"][test_period[-1]]
  #print(stock_data[stock]["Close"][test_period[-1]])

print('The money earned by LSTM with Fundamental and Technical data is ',trading)
print('The money earned by LSTM with Fundamental and Technical data is %',trading*100/original_money)
print('The money earned by LSTM with Fundamental and Technical data is %(unchange)',(end_money-original_money)*100/original_money)

Epoch 1/10
19/20 [===========================>..] - ETA: 0s - loss: 6.3923
Epoch 1: val_loss improved from inf to 4.44832, saving model to /content/drive/My Drive/FYP Data/weights_GRU.h5
20/20 [==============================] - 11s 272ms/step - loss: 6.2562 - val_loss: 4.4483 - lr: 0.0100
Epoch 2/10
19/20 [===========================>..] - ETA: 0s - loss: 2.7716
Epoch 2: val_loss improved from 4.44832 to 1.11747, saving model to /content/drive/My Drive/FYP Data/weights_GRU.h5
20/20 [==============================] - 1s 75ms/step - loss: 2.7163 - val_loss: 1.1175 - lr: 0.0100
Epoch 3/10
17/20 [========================>.....] - ETA: 0s - loss: 0.8413
Epoch 3: val_loss improved from 1.11747 to 0.21432, saving model to /content/drive/My Drive/FYP Data/weights_GRU.h5
20/20 [==============================] - 1s 77ms/step - loss: 0.7934 - val_loss: 0.2143 - lr: 0.0100
Epoch 4/10
18/20 [==========================>...] - ETA: 0s - loss: 0.4075
Epoch 4: val_loss did not improve from 0.21432
20/2

In [ ]:
stock_selection

[['VLO',
  'LOW',
  'NEM',
  'REGN',
  'ILMN',
  'URI',
  'BWA',
  'ADSK',
  'TTWO',
  'DAL',
  'ISRG',
  'ALK',
  'EQR',
  'MAR',
  'YUM',
  'MNST',
  'TER',
  'AAPL',
  'GPN',
  'RCL',
  'LEN',
  'CPRT',
  'PWR',
  'STX',
  'VRSN',
  'CBRE',
  'TYL',
  'WDC',
  'HUM',
  'FCX'],
 ['VLO',
  'LOW',
  'NEM',
  'REGN',
  'ILMN',
  'URI',
  'BWA',
  'ADSK',
  'TTWO',
  'DAL',
  'ISRG',
  'ALK',
  'EQR',
  'MAR',
  'YUM',
  'MNST',
  'TER',
  'AAPL',
  'GPN',
  'RCL',
  'LEN',
  'CPRT',
  'PWR',
  'STX',
  'VRSN',
  'CBRE',
  'TYL',
  'WDC',
  'HUM',
  'FCX'],
 ['VLO',
  'LOW',
  'NEM',
  'REGN',
  'ILMN',
  'URI',
  'BWA',
  'ADSK',
  'TTWO',
  'DAL',
  'ISRG',
  'ALK',
  'EQR',
  'MAR',
  'YUM',
  'MNST',
  'TER',
  'AAPL',
  'GPN',
  'RCL',
  'LEN',
  'CPRT',
  'PWR',
  'STX',
  'VRSN',
  'CBRE',
  'TYL',
  'WDC',
  'HUM',
  'FCX'],
 ['VLO',
  'LOW',
  'NEM',
  'REGN',
  'ILMN',
  'URI',
  'BWA',
  'ADSK',
  'TTWO',
  'DAL',
  'ISRG',
  'ALK',
  'EQR',
  'MAR',
  'YUM',
  'MNST',
  'TER'

#rolling strategies

In [ ]:
es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=10, verbose=1)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
mcp = ModelCheckpoint(filepath='/content/drive/My Drive/FYP Data/weights_GRU_fintune.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False)
predictions = []

for i in tqdm(range(26)):
  if i == 0:
    GRU_model = keras.models.load_model('/content/drive/My Drive/FYP Data/weights_GRU.h5')
  else:
    GRU_model = keras.models.load_model('/content/drive/My Drive/FYP Data/weights_GRU_fintune.h5')
  prediction = GRU_model.predict(x_test[i:i+1])
  predictions.append(prediction)
  x_train = X[i:train_point+i]
  y_train = Y[i:train_point+i]

  #fintune model
  history = GRU_model.fit(x_train, y_train, shuffle=True, epochs=10, callbacks=[es, rlr, mcp, tb], validation_split=0.2, verbose=1, batch_size=10)



  0%|          | 0/26 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step
Epoch 1/10
18/20 [==========================>...] - ETA: 0s - loss: 0.2734
Epoch 1: val_loss improved from inf to 0.21504, saving model to /content/drive/My Drive/FYP Data/weights_GRU.h5
20/20 [==============================] - 6s 132ms/step - loss: 0.2721 - val_loss: 0.2150 - lr: 0.0100
Epoch 2/10
19/20 [===========================>..] - ETA: 0s - loss: 0.2725
Epoch 2: val_loss did not improve from 0.21504
20/20 [==============================] - 0s 12ms/step - loss: 0.2720 - val_loss: 0.2169 - lr: 0.0100
Epoch 3/10
19/20 [===========================>..] - ETA: 0s - loss: 0.2675
Epoch 3: val_loss improved from 0.21504 to 0.21045, saving model to /content/drive/My Drive/FYP Data/weights_GRU.h5
20/20 [==============================] - 1s 72ms/step - loss: 0.2668 - val_loss: 0.2105 - lr: 0.0100
Epoch 4/10
19/20 [===========================>..] - ETA: 0s - loss: 0.2568
Epoch 4: val_loss improved from 0.21045 to 0.20054, saving model to /co

In [ ]:
predictions = np.vstack([predictions[i] for i in range(26)])

(26, 500)

In [ ]:
#Model Evaluation
predictions = scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print('rmse', rmse)

#profit test
test_period = time[n_past+train_point:]

#calculate the top 10 stock we picked
stock_selection_index = np.argsort(predictions)[:,-30:]
stock_selection = []
for index in stock_selection_index:
  stock_list = []
  for stock_i in index:
    stock_list.append(SP500_ticker[stock_i])
  stock_selection.append(stock_list)

#trading simulate
original_money = 0
trading = 0
for stock in stock_selection[0]:
  original_money = original_money + stock_data[stock]["Close"][test_period[0]]
  #print(stock_data[stock]["Close"][test_period[0]])


trading = - original_money
i = 0

for stock_list in stock_selection:
  if i==0 or 25:
    i = i + 1
    continue

  for stock in stock_list:
    trading = trading + stock_data[stock]["Close"][test_period[i-1]] - stock_data[stock]["Close"][test_period[i]]
    #print(trading)
  
  i = i + 1

end_money = 0

for stock in stock_selection[-1]:
  trading = trading + stock_data[stock]["Close"][test_period[-1]]
  end_money = end_money + stock_data[stock]["Close"][test_period[-1]]
  #print(stock_data[stock]["Close"][test_period[-1]])

print('The money earned by LSTM with Fundamental and Technical data is ',trading)
print('The money earned by LSTM with Fundamental and Technical data is %',trading*100/original_money)
print('The money earned by LSTM with Fundamental and Technical data is %(unchange)',(end_money-original_money)*100/original_money)

rmse 0.017849340602343614
The money earned by LSTM with Fundamental and Technical data is  2194.788293838501
The money earned by LSTM with Fundamental and Technical data is % 49.63684595275108
The money earned by LSTM with Fundamental and Technical data is %(unchange) 49.63684595275108


In [ ]:
stock_selection

[['VLO',
  'LOW',
  'NEM',
  'ILMN',
  'REGN',
  'URI',
  'BWA',
  'ADSK',
  'TTWO',
  'DAL',
  'ISRG',
  'ALK',
  'EQR',
  'MAR',
  'YUM',
  'MNST',
  'TER',
  'AAPL',
  'GPN',
  'RCL',
  'LEN',
  'CPRT',
  'PWR',
  'STX',
  'VRSN',
  'CBRE',
  'TYL',
  'WDC',
  'HUM',
  'FCX'],
 ['ABC',
  'TPR',
  'MTCH',
  'ALK',
  'TJX',
  'HUM',
  'PKI',
  'BKNG',
  'WDC',
  'AMZN',
  'VTR',
  'VRSN',
  'ILMN',
  'MAR',
  'EQIX',
  'TT',
  'CVS',
  'O',
  'MCO',
  'HPQ',
  'TSN',
  'AOS',
  'AMD',
  'GLW',
  'CNC',
  'AAPL',
  'URI',
  'VLO',
  'MNST',
  'INCY'],
 ['WYNN',
  'FDS',
  'PTC',
  'MU',
  'NVDA',
  'SBAC',
  'AKAM',
  'GRMN',
  'STLD',
  'KMX',
  'CI',
  'BIIB',
  'UNH',
  'LRCX',
  'FCX',
  'TRMB',
  'NEM',
  'NFLX',
  'MNST',
  'ADSK',
  'INCY',
  'ATVI',
  'VRSN',
  'CTSH',
  'PXD',
  'TER',
  'AAPL',
  'EW',
  'AMZN',
  'ILMN'],
 ['WYNN',
  'FDS',
  'PTC',
  'MU',
  'NVDA',
  'SBAC',
  'AKAM',
  'GRMN',
  'STLD',
  'KMX',
  'CI',
  'BIIB',
  'UNH',
  'LRCX',
  'FCX',
  'TRMB',
  'N

In [ ]:
df_stock_selection = pd.DataFrame(stock_selection)
df_stock_selection.to_csv("drive/My Drive/FYP Data/stock_selection_rolling.csv",index=True, header=True )

##fintune with past 4 years

In [ ]:
es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=10, verbose=1)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
mcp = ModelCheckpoint(filepath='/content/drive/My Drive/FYP Data/weights_GRU_rolling.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False)
predictions = []

for i in tqdm(range(26)):
  if i == 0:
    GRU_model = keras.models.load_model('/content/drive/My Drive/FYP Data/weights_GRU.h5')
  else:
    GRU_model = keras.models.load_model('/content/drive/My Drive/FYP Data/weights_GRU_rolling.h5')
  prediction = GRU_model.predict(x_test[i:i+1])
  predictions.append(prediction)
  x_train = X[train_point-4:train_point+i]
  y_train = Y[train_point-4:train_point+i]

  #fintune model
  history = GRU_model.fit(x_train, y_train, shuffle=True, epochs=10, callbacks=[es, rlr, mcp, tb], validation_split=0.2, verbose=1, batch_size=10)



  0%|          | 0/26 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step
Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.2333
Epoch 1: val_loss improved from inf to 0.19986, saving model to /content/drive/My Drive/FYP Data/weights_GRU_rolling.h5
1/1 [==============================] - 7s 7s/step - loss: 0.2333 - val_loss: 0.1999 - lr: 0.0100
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.2516
Epoch 2: val_loss improved from 0.19986 to 0.19802, saving model to /content/drive/My Drive/FYP Data/weights_GRU_rolling.h5
1/1 [==============================] - 1s 1s/step - loss: 0.2516 - val_loss: 0.1980 - lr: 0.0100
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.2382
Epoch 3: val_loss did not improve from 0.19802
1/1 [==============================] - 0s 52ms/step - loss: 0.2382 - val_loss: 0.1982 - lr: 0.0100
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 0.2153
Epoch 4: val_loss did not improve from 0.19802
1/1 [======================

In [ ]:
#Model Evaluation
predictions = np.vstack([predictions[i] for i in range(26)])
predictions = scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print('rmse', rmse)

#profit test
test_period = time[n_past+train_point:]

#calculate the top 10 stock we picked
stock_selection_index = np.argsort(predictions)[:,-30:]
stock_selection = []
for index in stock_selection_index:
  stock_list = []
  for stock_i in index:
    stock_list.append(SP500_ticker[stock_i])
  stock_selection.append(stock_list)

#trading simulate
original_money = 0
trading = 0
for stock in stock_selection[0]:
  original_money = original_money + stock_data[stock]["Close"][test_period[0]]
  #print(stock_data[stock]["Close"][test_period[0]])


trading = - original_money
i = 0

for stock_list in stock_selection:
  if i==0 or 25:
    i = i + 1
    continue

  for stock in stock_list:
    trading = trading + stock_data[stock]["Close"][test_period[i-1]] - stock_data[stock]["Close"][test_period[i]]
    #print(trading)
  
  i = i + 1

end_money = 0

for stock in stock_selection[-1]:
  trading = trading + stock_data[stock]["Close"][test_period[-1]]
  end_money = end_money + stock_data[stock]["Close"][test_period[-1]]
  #print(stock_data[stock]["Close"][test_period[-1]])

print('The money earned by LSTM with Fundamental and Technical data is ',trading)
print('The money earned by LSTM with Fundamental and Technical data is %',trading*100/original_money)
print('The money earned by LSTM with Fundamental and Technical data is %(unchange)',(end_money-original_money)*100/original_money)

rmse 0.0207318021662784
The money earned by LSTM with Fundamental and Technical data is  -782.1986141204834
The money earned by LSTM with Fundamental and Technical data is % -10.99240659100257
The money earned by LSTM with Fundamental and Technical data is %(unchange) -10.99240659100257


In [ ]:
stock_selection

[['AVB',
  'CNC',
  'NSC',
  'CI',
  'VTR',
  'MO',
  'ISRG',
  'CTSH',
  'AMZN',
  'EXR',
  'SPG',
  'ALGN',
  'TDG',
  'ROST',
  'JNPR',
  'NFLX',
  'DHI',
  'HUM',
  'ODFL',
  'ALK',
  'TER',
  'EW',
  'TJX',
  'PWR',
  'FFIV',
  'BKNG',
  'INCY',
  'ILMN',
  'URI',
  'AAPL'],
 ['SPG',
  'DGX',
  'CPRT',
  'JNPR',
  'BALL',
  'AMZN',
  'A',
  'AES',
  'AKAM',
  'TER',
  'NSC',
  'AVB',
  'FCX',
  'RMD',
  'CPT',
  'ODFL',
  'TPR',
  'BIO',
  'EW',
  'ALK',
  'GOOGL',
  'FFIV',
  'EXR',
  'BKNG',
  'PWR',
  'TJX',
  'DHI',
  'AAPL',
  'ALGN',
  'TDG'],
 ['CAG',
  'HOLX',
  'AMD',
  'GOOG',
  'FFIV',
  'ENPH',
  'ODFL',
  'MPWR',
  'GOOGL',
  'MGM',
  'RMD',
  'BALL',
  'CPRT',
  'NVDA',
  'NSC',
  'TJX',
  'DHI',
  'EW',
  'BKNG',
  'FCX',
  'EXR',
  'SBUX',
  'TPR',
  'PWR',
  'ALK',
  'AAPL',
  'AES',
  'BIO',
  'ALGN',
  'TDG'],
 ['CAG',
  'HOLX',
  'AMD',
  'GOOG',
  'FFIV',
  'ENPH',
  'ODFL',
  'MPWR',
  'GOOGL',
  'MGM',
  'RMD',
  'BALL',
  'CPRT',
  'NVDA',
  'NSC',
  'TJX',

In [ ]:
df_stock_selection = pd.DataFrame(stock_selection)
df_stock_selection.to_csv("drive/My Drive/FYP Data/stock_selection_rolling_4.csv",index=True, header=True )

In [ ]:
es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=10, verbose=1)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
mcp = ModelCheckpoint(filepath='/content/drive/My Drive/FYP Data/weights_LSTM_rolling.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False)
predictions = []

for i in tqdm(range(26)):
  if i == 0:
    LSTM_model = keras.models.load_model('/content/drive/My Drive/FYP Data/weights_w_fundamental_1.h5')
  else:
    LSTM_model = keras.models.load_model('/content/drive/My Drive/FYP Data/weights_LSTM_rolling.h5')
  prediction = LSTM_model.predict(x_test[i:i+1])
  predictions.append(prediction)
  x_train = X[train_point-4:train_point+i]
  y_train = Y[train_point-4:train_point+i]

  #fintune model
  history = LSTM_model.fit(x_train, y_train, shuffle=True, epochs=10, callbacks=[es, rlr, mcp, tb], validation_split=0.2, verbose=1, batch_size=10)



  0%|          | 0/26 [00:00<?, ?it/s]

1/1 [==============================] - 1s 649ms/step
Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.3160
Epoch 1: val_loss improved from inf to 0.20345, saving model to /content/drive/My Drive/FYP Data/weights_LSTM_rolling.h5
1/1 [==============================] - 4s 4s/step - loss: 0.3160 - val_loss: 0.2035 - lr: 0.0100
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.2828
Epoch 2: val_loss did not improve from 0.20345
1/1 [==============================] - 0s 45ms/step - loss: 0.2828 - val_loss: 0.2126 - lr: 0.0100
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.2998
Epoch 3: val_loss did not improve from 0.20345
1/1 [==============================] - 0s 43ms/step - loss: 0.2998 - val_loss: 0.2199 - lr: 0.0100
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 0.2648
Epoch 4: val_loss did not improve from 0.20345
1/1 [==============================] - 0s 44ms/step - loss: 0.2648 - val_loss: 0.2312 - lr: 0.0100

In [ ]:
#Model Evaluation
predictions = np.vstack([predictions[i] for i in range(26)])
predictions = scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print('rmse', rmse)

#profit test
test_period = time[n_past+train_point:]

#calculate the top 10 stock we picked
stock_selection_index = np.argsort(predictions)[:,-30:]
stock_selection = []
for index in stock_selection_index:
  stock_list = []
  for stock_i in index:
    stock_list.append(SP500_ticker[stock_i])
  stock_selection.append(stock_list)

#trading simulate
original_money = 0
trading = 0
for stock in stock_selection[0]:
  original_money = original_money + stock_data[stock]["Close"][test_period[0]]
  #print(stock_data[stock]["Close"][test_period[0]])


trading = - original_money
i = 0

for stock_list in stock_selection:
  if i==0 or 25:
    i = i + 1
    continue

  for stock in stock_list:
    trading = trading + stock_data[stock]["Close"][test_period[i-1]] - stock_data[stock]["Close"][test_period[i]]
    #print(trading)
  
  i = i + 1

end_money = 0

for stock in stock_selection[-1]:
  trading = trading + stock_data[stock]["Close"][test_period[-1]]
  end_money = end_money + stock_data[stock]["Close"][test_period[-1]]
  #print(stock_data[stock]["Close"][test_period[-1]])

print('The money earned by LSTM with Fundamental and Technical data is ',trading)
print('The money earned by LSTM with Fundamental and Technical data is %',trading*100/original_money)
print('The money earned by LSTM with Fundamental and Technical data is %(unchange)',(end_money-original_money)*100/original_money)

rmse 0.0024281933089145102
The money earned by LSTM with Fundamental and Technical data is  -287.01530742645264
The money earned by LSTM with Fundamental and Technical data is % -6.645764532413249
The money earned by LSTM with Fundamental and Technical data is %(unchange) -6.645764532413249
